In [1]:
import preprocess as pp
import configs
import persister
import relation
import lda
import vis
import evaluate
import embedding
from utils import *

from collections import Counter
from pprint import pprint
from functools import reduce

import re
import numpy as np
from nltk.tree import Tree
from nltk import word_tokenize,sent_tokenize

not use stanford CoreNLP client!


In [2]:
is_news = 1

l = 7
t = 4
m = "c_v"
i = 200
min_df = 1

# is_news = 0
# l = 6
# t = 14
# m = "c_v"
# i = 200
# min_df = 1

size = 100

# load
if is_news:
#     _raw = persister.load_json(configs.RAWNEWS)
    parse = persister.read_parse()
    _input = persister.read_input(configs.NEWSINPUT)
    model_name = configs.NEWSMODEL.format(l, t, m, i, min_df)
    terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA.format(model_name))
    model = persister.load_model(model_name)
    vec = persister.load_model(configs.NEWSVEC.format(min_df))
    w2vmodel = persister.load_wv(configs.NEWSWV.format(size))
    print("load news")
else:
    _raw = persister.load_json(configs.RAWABSTRACT)
    _input = persister.read_input(configs.ABSTRACTINPUT)
    model_name = configs.ABSTRACTMODEL.format(l, t, m, i, min_df)
    terms, doc_topic, topic_word = persister.read_lda(configs.ABSTRACTLDA.format(model_name))
    parse = persister.read_parse(configs.ABSTRACTPARSE)
    model = persister.load_model(model_name)
    vec = persister.load_model(configs.ABSVEC.format(min_df))
    w2vmodel = persister.load_wv(configs.ABSWV.format(size))
    print("load abs")
tf = vec.fit_transform(_input)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic,20)
word_values = []
for i in top_terms:
    tmp = []
    for j in i:
        tmp.append((terms[j[0]], j[1]))
#         tmp.append(terms[j[0]])
    word_values.append(tmp)
df_top_words, df_top_docs = lda.pd_topics_vis(word_values, top_docs)

load news


In [3]:
len(terms)

15558

In [4]:
distr = lda.get_dominant_topic(doc_topic)
print("docs distribution:",dict(Counter(distr)))

docs distribution: {1: 916, 3: 457, 0: 542, 2: 85}


In [5]:
print("top terms info:")
df_top_words

top terms info:


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,"(gun, 959.45)","(write, 603.99)","(people, 568.68)","(article, 445.17)","(make, 435.45)","(use, 394.72)","(say, 386.98)","(think, 369.75)","(weapon, 362.1)","(law, 316.42)","(know, 293.25)","(time, 289.84)","(state, 282.71)","(like, 274.26)","(right, 273.93)","(firearm, 257.43)","(government, 256.04)","(kill, 244.17)","(thing, 237.24)","(believe, 216.95)"
Topic 1,"(car, 844.34)","(write, 781.44)","(use, 548.6)","(article, 533.01)","(know, 423.93)","(like, 412.45)","(look, 395.38)","(think, 322.15)","(good, 286.36)","(need, 275.5)","(time, 273.04)","(drive, 270.33)","(want, 266.02)","(thanks, 264.66)","(say, 262.86)","(make, 259.05)","(file, 257.14)","(image, 236.91)","(buy, 232.26)","(work, 229.65)"
Topic 2,"(water, 106.34)","(engine, 105.76)","(oil, 92.31)","(op, 57.58)","(compass, 52.76)","(dept, 43.91)","(air, 39.65)","(install, 39.64)","(tire, 38.31)","(ford, 37.49)","(sensor, 37.08)","(new, 36.28)","(col, 35.63)","(mwra, 34.49)","(drain, 34.2)","(quicktime, 33.69)","(fuel, 31.78)","(liter, 30.33)","(fp, 29.87)","(change, 29.61)"
Topic 3,"(write, 383.34)","(article, 346.02)","(food, 240.85)","(know, 225.4)","(msg, 209.49)","(use, 208.58)","(people, 189.83)","(like, 182.0)","(say, 176.23)","(think, 173.29)","(disease, 167.98)","(make, 165.32)","(time, 160.41)","(doctor, 156.86)","(cause, 152.42)","(patient, 140.71)","(effect, 136.78)","(science, 131.73)","(study, 131.39)","(work, 126.59)"


In [6]:
print("top docs info:")
df_top_docs

top docs info:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Topic 0,"(1810, 0.999)","(1584, 0.999)","(1701, 0.999)","(1812, 0.999)","(1500, 0.998)","(1927, 0.997)","(1801, 0.997)","(1558, 0.997)","(1798, 0.997)","(1900, 0.997)","(1800, 0.997)","(1993, 0.997)","(1615, 0.997)","(1964, 0.996)","(1664, 0.996)","(1641, 0.996)","(1567, 0.996)","(1523, 0.996)","(1504, 0.996)","(1898, 0.996)"
Topic 1,"(211, 0.999)","(207, 0.999)","(70, 0.998)","(87, 0.998)","(399, 0.997)","(260, 0.997)","(927, 0.996)","(660, 0.996)","(64, 0.996)","(2, 0.996)","(922, 0.996)","(157, 0.996)","(328, 0.996)","(814, 0.996)","(723, 0.996)","(19, 0.996)","(457, 0.996)","(530, 0.996)","(512, 0.995)","(222, 0.995)"
Topic 2,"(178, 0.998)","(1981, 0.963)","(1569, 0.925)","(1095, 0.917)","(780, 0.833)","(854, 0.825)","(673, 0.773)","(1808, 0.773)","(542, 0.746)","(36, 0.742)","(1459, 0.711)","(507, 0.703)","(943, 0.702)","(817, 0.697)","(679, 0.692)","(538, 0.69)","(830, 0.686)","(907, 0.671)","(704, 0.656)","(757, 0.65)"
Topic 3,"(1278, 0.999)","(1490, 0.998)","(1021, 0.998)","(1001, 0.998)","(139, 0.997)","(1195, 0.997)","(1436, 0.997)","(1151, 0.997)","(1217, 0.996)","(1008, 0.996)","(1396, 0.996)","(1297, 0.996)","(1193, 0.996)","(1276, 0.996)","(1137, 0.995)","(1002, 0.995)","(1498, 0.995)","(1017, 0.995)","(1346, 0.995)","(1314, 0.995)"


In [7]:
# vis.pyLDA(topic_word, doc_topic, [len(s) for s in [word_tokenize(corp) for corp in _input]], vec.get_feature_names(), np.array(sum(tf).todense())[0])

In [8]:
t_idx = 1
for top_doc in top_docs[t_idx]:
    _idx = top_doc[0]
    if type(parse[_idx]) is str:
        print(_idx, "parse", "err")
        continue
    sents = relation.convert_parse2lemma_sents(parse[_idx])
    sort_idxs, importance, counts = relation.extract_important_sents(sents, [terms[x[0]] for x in top_terms[t_idx]], [x[1] for x in top_terms[t_idx]])
    for i in sort_idxs:
        if importance[i]>0:
            print(round(importance[i], 2), counts[i], _idx, i)
            sent_tokens = parse[_idx]["sentences"][i]["tokens"]
            sent_deps = parse[_idx]["sentences"][i]["enhancedPlusPlusDependencies"]
            print(" ".join([s["lemma"] for s in sent_tokens]))
#             sents = sent_tokenize(_raw[_idx])
#             if len(sents) > i:
#                 print(sents[i])
#             else:
#                 print("sent token differ")
            triples = relation.extract_triples_from_sent(sent_deps, sent_tokens, True)
            for triple in triples:
                if None in triple:
                    print(triple)
                    continue
                s = [sent_tokens[i]["originalText"] for i in triple[0]]
                p = [sent_tokens[i]["originalText"] for i in triple[1]]
                o = [sent_tokens[i]["originalText"] for i in triple[2]]
                print("{}-{}-{}".format(s,p,o))
            print("="*40)

1.2 2 211 71
the ` write ' menu be much like ` read ' menu , but only allow you to change directory .
['write', 'menu']-['is']-['read', 'but', 'allow']
1.01 4 211 103
this program will ask you some question if you want to write image to file .
['program']-['ask']-['questions']
[(8,), (9,), None]
['you']-['write']-['image']
[(8,), (14,), None]
0.94 2 211 70
if you move or copy file , you will enter ` write ' menu .
[(7,), (9,), None]
['you']-['move']-['files']
[(1,), (4,), None]
['you']-['write']-['menu']
0.91 2 211 10
introduction : this program can let you read , write and display image with different format .
0.9 1 211 125
information : Program write by Jih-Shin Ho .
0.9 1 211 137
also I will thank the author who write tiff and Jpeg library .
['I']-['thank']-['authors']
0.9 1 211 63
write : gif , Sun Raster , Jpeg , XBM , pbm , PGM , PPM , PM , Tiff , Targa , XPM , Mac Paint , Ascii , Laser Jet , iff/ilbm , Window bmp , + Mac PICT , VIS , fit , face , PCX , gem/img .
0.9 1 211 121
th

0.97 2 260 0
: Robert G. Carpenter write : : Hi Netters , : : I be build a cad package and need a 3d graphic library that can handle : some rudimentry task , such as hidden line removal , shade , animation , etc. : : can you please offer some recommendation ?
[(1, 2, 3), (4,), None]
0.96 2 260 16
do not you always want to write this program that visualize the structure of three-dimensional molecule ?
[(2,), (4,), None]
['you']-['write']-['program']
0.9 1 260 17
and do not the task of write you 3d conversion routine keep you from actually do it ?
['And', 'task']-['keep']-['you']
['And', 'task']-['doing']-['it']
0.83 2 260 6
I have never use this package , nor do I know anyone who do , but the price be right :--rrb- Hope this help .
['I']-['used']-['package']
0.57 2 260 32
however , I feelt that the way it be implement now make it easier to use for a lot more people than the select ` oop-guild ' .
['I']-['feelt']-['that']
0.52 1 260 11
the Docs now contain a chapter for c programmer on h

0.07 1 660 12
-lrb- read Remar Sutton 's `` do not get take every time '' for a list . -rrb-
0.07 1 660 1
the finance guy then bring in the manager on duty who proceed to give I a hard time .
['finance', 'guy']-['brought', 'in']-['manager']
['finance', 'guy']-['brought', 'in']-['duty']
0.06 1 660 8
we go to a Mazda dealership and describe what `` we '' want .
['We']-['went', 'to']-['Mazda', 'dealership']
0.05 1 660 22
we say that we will not bother he anymore , we be go next door to the Nissan dealership .
[(0,), (1,), None]
['we']-['bother']-['him']
0.05 1 660 4
needless to say , they be not please by the turn of event .
[(4,), (7,), None]
[(4,), (0,), None]
0.05 1 660 3
he make some smart remark so I tell he where he could stick it , snatch back my check and leave .
['He']-['made']-['smart', 'remark']
['I']-['told']-['him']
['he']-['stick']-['it']
['he']-['snatched']-['check']
[(10,), (20,), None]
0.0 1 660 26
so my friend buy a Sentra .
['friend']-['bought']-['Sentra']
0.0 1 660 25


0.15 1 814 32
if you think so you sure do not pay attention to my posting .
[(1,), (2,), None]
['you']-['pay']-['to', 'postings']
0.07 1 814 2
they drive goddamn rabbit at 120 mph in Europe , pal , and I reckon a Taurus be at least as capable as a rabbit .
['They']-['drive']-['goddamn', 'Rabbits']
['They']-['drive']-['at', 'MPH', 'in', 'Europe']
0.05 1 814 19
this make they far more predictable than most US highway .
[(0,), (1,), None]
[(2,), (5,), None]
0.05 1 814 31
you have you mind make up already .
[(0,), (1,), None]
[(3,), (4,), None]
1.45 3 723 0
in a previous article , a207706@moe.dseg.ti.com say : in article c5l8re.28@constellation.ecn.uoknor.edu callison@uokmax.ecn.uoknor.edu write : in article 1993apr15.232412.2261@ganglion.ann-arbor.mi.us david@ganglion.ann-arbor.mi.us write : in article 5214@unisql.uucp wrat@unisql.uucp write : in article chrissc587qb.d1b@netcom.com chriss@netcom.com write : why anyone would order a sho with a automatic transmission be beyond I ; if you ca

in article 30523@hacgate.scg.hac.com lee@luke.rsg.hac.com write : the original post complain about SGI come out with newer -lrb- and better -rrb- architecture and not have a upgrade path from the older one , and that DEC do .
0.49 3 222 12
and every time I look at it , I be go to think about SGI and how I could have better spend my money -lrb- actually * you * money , since we be support almost entirely by Federal tax dollar -rrb- .
[(8,), (10,), None]
['I']-['think', 'about']-['SGI', 'and', 'have']
0.27 1 222 11
I be go to be look at this box for the next five year .
[(0,), (2,), None]
['I']-['looking', 'at']-['next', 'years']
0.15 1 222 13
now you will have to pardon I while I go off and hiss and fume in a corner somewhere and think dark , libelous thought .
[(1,), (3,), None]
['you']-['pardon']-['me']
[(8,), (9,), None]
['I']-['hiss']-['in', 'corner']
['I']-['think']-['dark', 'libelous', 'thoughts']
0.08 2 222 10
that mean that I need to buy the best piece of equipment that I can wh

In [9]:
extended = relation.extend_lda_results(parse, _input, top_terms, top_docs, terms, "originalText", top_n=20, score_method="tf")
print([len(t) for t in extended])
for triple in extended[t_idx]:
    print(round(triple[1],4),"{}-[{}]-{}".format(" ".join(triple[0][0])," ".join(triple[0][1])," ".join(triple[0][2])))

[20, 20, 13, 20]
93.6767 you-[write]-image
89.0 Sure |-[talked]-price for car
89.0 Sure |-[talked]-price for car
84.8625 jk87377@lehtori.cc.tut.fi-[writes]-In article
84.8625 1993Apr5.203719@usho0b.hou281.chevron.com hhtra@usho0b.hou281.chevron.com-[writes]-In article
84.8625 andreasa@dhhalden.no-[writes]-In article andreasa.157.735211806@dhhalden.no
84.8625 rchui@nswc-wo.nswc.navy.mil-[writes]-In article 1993Mar17.185725.13487@relay.nswc.navy.mil
84.8625 fredd@shuksan-[writes]-In article C4vIr5.L3r@shuksan.ds.boeing.com
84.8625 1993Apr15.161531.11737@newsgate.sps.mot.com rapw20@email.sps.mot.com-[writes]-In article
79.8928 write menu-[is]-read but allow
76.0 cars-[be]-better deals
76.0 cars-[be]-better deals
69.0 highway car-[squeezed by]-sets
67.3254 you-[write]-program
67.0 Stuff for cars-[be]-expensive
64.0 car-[eats]-set
63.0 typical mileage for cars-[be]-km
59.0 you-[paying]-for same car
59.0 you-[paying]-for same car
58.0 Saturn-[selling]-car


In [10]:
ws = 30
measure = "c_uci"
new_coherence = []
old = lda.get_coherence(tf, terms, topic_word, _input, measure, top_n=20, window_size=ws)
emethods = ["basic", "w2v", "tf", "tfidf"]
for e in emethods:
    extended = relation.extend_lda_results(parse, _input, top_terms, top_docs, terms, "lemma", top_n=20, score_method=e)
    new = evaluate.evaluate_triples_by_coherence(extended,vec,terms, topic_word,_input,top_terms,top_docs,measure, window_size=ws, score="tf", add_old=True)
    print(e, round(np.mean(new),2))
    new_coherence.append(new)

OSError: [Errno 22] Invalid argument

In [ ]:
list(enumerate([1,2,3]))

In [ ]:
evaluate.draw_coherence_line(old, new_coherence, emethods, measure=measure, ws=ws)

In [ ]:
ps = relation.get_phrases_by_pattern(parse)
sorted(dict(Counter([" ".join(x) for x in ps])).items(), key=lambda x:x[1], reverse=True)

In [ ]:
vis.draw_word_cloud("", {terms[k]:v for k,v in top_terms[t_idx]}, False)